# CAES Dataset Notebook
## 1. Tagging

Related to the CAES essay dataset.

In [4]:
import pandas as pd

df = pd.read_csv("data/caes.raw.csv")
df

,estudiante,tarea,tipologia,tema,edad,sexo,pais,l1,estudios,edad_inicio,meses_estudio,contactos_habla_es,nivel,oraciones,oraciones_longitud,tokens,texto
0,417.0,1156.0,Carta,Carta amigo,26.0,Mujer,Brasil,Portugués,Universidad,25.0,8.0,Amigos,B1,"['Hola Carlos !', 'Hace mucho que no te veo ho...",18,212,Hola Carlos ! Hace mucho que no te veo hombre ...
1,1387.0,3934.0,Carta,Carta amigo,20.0,Mujer,Marruecos,Árabe,Universidad,15.0,30.0,No,A2,"['Hola querida Sara ,', 'espero que estes muy ...",9,175,"Hola querida Sara , espero que estes muy bien ..."
2,2273.0,6370.0,Postal,Postal vacaciones,18.0,Mujer,Otro,Ruso,Universidad,18.0,8.0,No,A1,"['Como estais ?', '!', 'Hola amigos ! ?', 'Esp...",13,104,Como estais ? ! Hola amigos ! ? Espero que tod...
3,1522.0,4307.0,Postal,Postal vacaciones,22.0,Hombre,China,Chino mandarín,Universidad,21.0,2.0,No,A2,"['Cuando llegué a el destino , le propuso que ...",12,135,"Cuando llegué a el destino , le propuso que si..."
4,775.0,2159.0,Correo electrónico,Familia,27.0,Mujer,Francia,Francés,Otros,20.0,6.0,Amigos,A1,"['¿ Que tal ?', 'un beso', 'Espero que todo va...",8,80,¿ Que tal ? un beso Espero que todo va bien po...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30995,2366.0,6592.0,Carta,Carta amigo,19.0,Mujer,China,Chino mandarín,Universidad,7.0,70.0,Otros,B1,"['Buenas tardes !', '29_de_mayo_de_2019 salama...",8,299,Buenas tardes ! 29_de_mayo_de_2019 salamanca Q...
30996,1497.0,4236.0,Carta,Solicitud admisión,33.0,Hombre,China,Chino mandarín,Universidad,27.0,12.0,Amigos&Familiares,B2,['Tengo la seguridad de mí mismo para contener...,13,157,Tengo la seguridad de mí mismo para contener e...
30997,1538.0,4350.0,Correo electrónico,Reclamación compañía aérea,21.0,Hombre,China,Chino mandarín,Primaria,19.0,24.0,Amigos,B1,"['Distinguidos señor / señra :', 'Hola , geren...",15,178,"Distinguidos señor / señra : Hola , gerente . ..."
30998,2122.0,5976.0,Carta,Solicitud admisión,19.0,Mujer,Portugal,Portugués,Universidad,15.0,36.0,No,B2,['Le escribo esta carta porque me gustaría inm...,14,303,Le escribo esta carta porque me gustaría inmen...


In [7]:
# clean dataset
df = df.rename(
    columns={
        "texto": "full_text",
        "nivel": "score",
        "tema":  "prompt_name"
    }
)[["full_text", "score", "prompt_name"]]

# insert essay ID column
df.insert(0, "essay_id", range(len(df)))

# get word count
df["word_count"] = df["full_text"].str.split().str.len().astype(int)

# organize columns
df = df[["essay_id", "full_text", "score", "word_count", "prompt_name"]]

# remove nan
df.dropna(subset=['full_text', 'score'], inplace=True)

# lowercase and filter
df['full_text'] = df['full_text'].str.lower()
df = df[(df['word_count'] >= 80) & (df['word_count'] <= 850)].copy()
df.reset_index(drop=True, inplace=True)

# write to csv
df.to_csv("caes/cleaned_caes.csv", index=False)

df.info

<bound method DataFrame.info of        essay_id                                          full_text score  \
0             0  hola carlos ! hace mucho que no te veo hombre ...    B1   
1             1  hola querida sara , espero que estes muy bien ...    A2   
2             2  como estais ? ! hola amigos ! ? espero que tod...    A1   
3             3  cuando llegué a el destino , le propuso que si...    A2   
4             4  ¿ que tal ? un beso espero que todo va bien po...    A1   
...         ...                                                ...   ...   
26648     26648  buenas tardes ! 29_de_mayo_de_2019 salamanca q...    B1   
26649     26649  tengo la seguridad de mí mismo para contener e...    B2   
26650     26650  distinguidos señor / señra : hola , gerente . ...    B1   
26651     26651  le escribo esta carta porque me gustaría inmen...    B2   
26652     26652  me gusta ir a mallorca y me gusta espeicalment...    A1   

       word_count                 prompt_name  
0      